# Prepare EMG & MMG example data for explanation on webiste
See website "The Study"

In [1]:

import sys
import os
import pandas as pd
import numpy as np
import biosignalsnotebooks as bsnb
import biosignalsnotebooks.signal_samples as bsnb_ss
import re
import pickle
import scipy as sp
from datetime import datetime, timedelta

sys.path.append('../utils')

# importing
from feature_extraction_functions import *

## Prepare example for illustrating pre-processing

In [12]:
person='Johanna'
if person =='Amanda':
    file = '../../../data//raw_data_new/AB/ABennetts_opensignals_98D311FDFF90_2024-07-24_21-40-26.h5'
if person == 'Johanna':
    file = '../../../data/raw_data_new/JE/JE_2024-06-30_02-03-18.h5'


In [13]:
data, header = bsnb_ss.load(file, get_header=True)
sr = header["sampling rate"] # Sampling rate
resolution = header["resolution"] 
signal_emg = data['CH1']
signal_mmg = data['CH2']


In [14]:
# get time
time = bsnb.generate_time(signal_emg,sr)


In [15]:
# apply burst detection algorithm
burst_begin, burst_end = bsnb.detect_emg_activations(signal_emg,sr, smooth_level=20, threshold_level=10, 
                                                    time_units=True)[:2]
                                            
startTabata = burst_begin[burst_begin>30][0]  -30

In [16]:
# downsample signal for faster loading on website
signal_emg = signal_emg[::50]
signal_mmg = signal_mmg[::50]
time = time[::50]


In [17]:
emg_raw = pd.DataFrame([time,signal_emg]).T.rename(columns={0:'Time',1:'Signal'})
emg_raw['Signal type'] ='EMG'

mmg_raw = pd.DataFrame([time,signal_mmg]).T.rename(columns={0:'Time',1:'Signal'})
mmg_raw['Signal type'] = 'MMG'
raw = pd.concat([emg_raw,mmg_raw])
raw['Processing stage'] = 'Raw signal'

In [18]:
# Baseline shift.
signal_emg_base = np.array(signal_emg) - np.mean( signal_emg)
#emg_base = pd.DataFrame([time,signal_emg_base]).T.rename(columns={0:'Time',1:'Signal'})
#emg_base['Signal type'] = 'EMG'

signal_mmg_base = np.array(signal_mmg) - np.mean( signal_mmg)
#mmg_base = pd.DataFrame([time,signal_mmg_base]).T.rename(columns={0:'Time',1:'Signal'})
#mmg_base['Signal type'] = 'MMG'

#base = pd.concat([emg_base,mmg_base])
#base['stage'] = 'base'

# create bandpass filter for EMG of order 4
signal_emg_bandpass = bsnb.bandpass(signal_emg_base,f1=20,f2=450,  order=4, fs=1000,use_filtfilt=True)
emg_bandpass = pd.DataFrame([time,signal_emg_bandpass]).T.rename(columns={0:'Time',1:'Signal'})
emg_bandpass['Signal type'] = 'EMG'
signal_mmg_bandpass = bsnb.bandpass(signal_mmg_base,f1=2,f2=50,  order=4, fs=1000,use_filtfilt=True)
mmg_bandpass = pd.DataFrame([time,signal_mmg_bandpass]).T.rename(columns={0:'Time',1:'Signal'})
mmg_bandpass['Signal type'] = 'MMG'

bandpass= pd.concat([emg_bandpass,mmg_bandpass])
bandpass['Processing stage'] = 'Bandpass filter'



In [19]:
# rectification
signal_emg_rect = np.abs(signal_emg_bandpass)
emg_rect = pd.DataFrame([time,signal_emg_rect]).T.rename(columns={0:'Time',1:'Signal'})
emg_rect['Signal type'] = 'EMG'

signal_mmg_rect = np.abs(signal_mmg_bandpass)
mmg_rect = pd.DataFrame([time,signal_mmg_rect]).T.rename(columns={0:'Time',1:'Signal'})
mmg_rect['Signal type'] ='MMG'

rect = pd.concat([emg_rect, mmg_rect])
rect['Processing stage'] = 'Rectified signal'

In [20]:
# amplitude normalization
signal_emg_norm = signal_emg_rect/np.max(signal_emg_rect)
emg_norm = pd.DataFrame([time,signal_emg_norm]).T.rename(columns={0:'Time',1:'Signal'})
emg_norm['Signal type'] = 'EMG'

signal_mmg_norm = signal_mmg_rect/np.max(signal_mmg_rect)
mmg_norm = pd.DataFrame([time,signal_mmg_norm]).T.rename(columns={0:'Time',1:'Signal'})
mmg_norm['Signal type'] ='MMG'

norm = pd.concat([emg_norm, mmg_norm],axis=0)
norm['Processing stage'] = 'Normalized signal'

In [21]:



raw.to_csv('/Users/htr365/no_icloud/quantified_self_all/quantified_self/website/docs/data/'+person.lower()+'_raw_emg_example.csv')
bandpass.to_csv('/Users/htr365/no_icloud/quantified_self_all/quantified_self/website/docs/data/'+person.lower()+'_bandpass_emg_example.csv')
rect.to_csv('/Users/htr365/no_icloud/quantified_self_all/quantified_self/website/docs/data/'+person.lower()+'_rect_emg_example.csv')
norm.to_csv('/Users/htr365/no_icloud/quantified_self_all/quantified_self/website/docs/data/'+person.lower()+'_norm_emg_example.csv')

# Prepare examples for illustration of feature extraction

In [2]:
df = pd.read_pickle('/Users/htr365/no_icloud/quantified_self_all/data/data_pre_processed/AB/2024-06-25_AM.pickle')

In [3]:
sr=1000
frame=500
step=250
df_features_emg = features_estimation(df['signal_emg'], '', sr, frame,step, plot=False)[0]
df_features_emg = df_features_emg.transpose()


df_features_mmg = features_estimation(df['signal_mmg'], '', sr, frame, step, plot=False)[0]
df_features_mmg = df_features_mmg.transpose()
#df_features = pd.concat([df_features_emg,df_features_mmg.drop(['START','END'],axis=1)],axis=1)

In [4]:
df_features_emg = df_features_emg.rename(columns={'VAR_':'Variance', 'RMS_':'Root Mean Square', 'IEMG_': 'Integral', 'MAV_': 'Mean Absolute Value',\
                                'LOG_':'Log Detector', 'WL_':'Wavelength','ACC_':'Average Amplitude Change','DASDV_':'Difference Absolute Standard Deviation',\
                                    'WAMP_': 'Willison Amplitude', 'MYOP_':'Myopulse Percentage Rate','FR_':'Frequency Ratio', 'MNP_':'Mean Power','TP_':'Total Power',\
                                        'MNF_':'Mean Frequency','MDF_': 'Median Frequency','WENT_':'Wavelet Energy'})

df_features_mmg = df_features_mmg.rename(columns={'VAR_':'Variance', 'RMS_':'Root Mean Square', 'IEMG_': 'Integral', 'MAV_': 'Mean Absolute Value',\
                                'LOG_':'Log Detector', 'WL_':'Wavelength','ACC_':'Average Amplitude Change','DASDV_':'Difference Absolute Standard Deviation',\
                                    'WAMP_': 'Willison Amplitude', 'MYOP_':'Myopulse Percentage Rate','FR_':'Frequency Ratio', 'MNP_':'Mean Power','TP_':'Total Power',\
                                        'MNF_':'Mean Frequency','MDF_': 'Median Frequency','WENT_':'Wavelet Energy'})

In [5]:
df_features_emg_norm = df_features_emg.drop(['START','END'],axis=1) / df_features_emg.drop(['START','END'],axis=1).max(axis=0)
df_features_emg = pd.concat([df_features_emg[['START','END']],df_features_emg_norm],axis=1)

df_features_mmg_norm = df_features_mmg.drop(['START','END'],axis=1) / df_features_mmg.drop(['START','END'],axis=1).max(axis=0)
df_features_mmg = pd.concat([df_features_mmg[['START','END']],df_features_mmg_norm],axis=1)

In [6]:
df_features_emg_long = pd.melt(df_features_emg, id_vars=['START','END'],
                    var_name='variable', value_name='value')
df_features_emg_long['Signal Type'] ='EMG'
df_features_mmg_long = pd.melt(df_features_mmg, id_vars=['START','END'],
                    var_name='variable', value_name='value')
df_features_mmg_long['Signal Type'] ='MMG'


In [7]:
df_features_long = pd.concat([df_features_emg_long,df_features_mmg_long],axis=0)

In [8]:
df_features_long.to_csv('/Users/htr365/no_icloud/quantified_self_all/quantified_self/website/docs/data/amanda_features_example.csv')